# TGP Stage 1 analysis

This notebook performs an interactive analysis of one of the **Stage 1** datasets.
It starts by loading the raw data and proceeds with pre-processing and analysis steps, where each step is visualized.
The goal of this analysis is to extract an interesting region in the magnetic field ($B$) and plunger voltage ($V$) space where the device is both gapped and has zero-bias peaks (ZBPs).
The extracted regions with gapped ZPBs are where Stage 2 measurements will be performed.

In [ ]:
from pathlib import Path

import tgp
import xarray as xr

tgp.plot.set_mpl_rc()

folder = Path("..") / "data"

print("Using tgp code version:", tgp.__version__)

# Analysis on experimental data

We load the raw data and prepare it for the `tgp` code's analysis.
The `prepare` function performs renames and joins the left and right cutter arrays into a single index (if required).

In [ ]:
ds = xr.load_dataset(folder / "experimental" / "deviceA_stage1.nc", engine="h5netcdf")
ds = tgp.prepare.prepare(ds)

The dataset we have just loaded contains many data variables.

The following are the *raw* measured complex arrays
- `I_1w_L`, `I_1w_R`: the local $1\omega$ component of the current (corresponds to the local differential conductance, which the 1st derivative of current with respect to voltage) on the left and right
- `I_2w_L` and `I_2w_R`: the local $2\omega$ componentof the  current (2nd derivative of current with respect to voltage) on the left and right
- `I_2w_RL`, `I_2w_LR`: the non-local $2\omega$ component of the current on the left and right
- `I_3w_L`, `I_3w_R`: the local $3\omega$ component of the current (3rd derivative of current with respect to voltage) on the left and right

These are converted to the following arrays (*used in the analysis*) by only keeping the real part
- `L_2w_nl`, `R_2w_nl`: the non-local $2\omega$ component of the current on the left and right
- `L_3w`, `R_3w`: the local $3\omega$ component of the current on the left and right

We use $3\omega$ local signals to identify ZBP peaks and $2\omega$ non-local signals to distinguish gapped and gapless states. 

## Visualize the data

We can view the $1\omega$ signals where we plot the real parts of the conductance with magnetic field on the x-axis and plunger voltage on the y-axis.
Every separate plot is for a different cutter value pair.
First we show the conductance on the left then on the right.

In [ ]:
lim_1w = 1.0

tgp.one.print_lim(ds.L_1w, lim_1w)
tgp.one.print_lim(ds.R_1w, lim_1w)

# With no labels and titles:
# tgp.plot.one.plot_1w(ds, lim_1w, plot_kwargs=dict(figsize=(12, 8)), minimal=True)

tgp.plot.one.plot_1w(ds, lim_1w)

Similar to the plots above, we plot the $2\omega$ nonlocal signals.

In [ ]:
lim_2w = 1e3

tgp.one.print_lim(ds.L_2w_nl, lim_1w)
tgp.one.print_lim(ds.R_2w_nl, lim_1w)

tgp.plot.one.plot_2w(ds, lim_2w)

In [ ]:
tgp.plot.one.plot_2w_at(ds, cutter_pair_index=0, vmax=lim_2w)  # change the cutter_pair_index to see different plots

## Extract the gap from $2\omega$ non-local signal

After we have inspected the data, we are ready to perform the first analysis step.
The $2\omega$ component of the conductance informs us about the energy gap of the spectrum.
We threshold the non-local $2\omega$ component above its noisy background to get a boolean array of the gap.
This works because the non-local conductance is suppressed below the gap.

In [ ]:
# Manually set the threshold region
# tgp.one.set_2w_th(ds, B_max=0.7, V_max=-1.410, verbose=True)

# Automatically set
tgp.one.set_2w_th(ds)

The result is an array with the dimensions `(cutter_pair_index, B, V)`.
We can visualize this by plotting the gap per cutter pair value.

In [ ]:
tgp.plot.one.plot_2w_th(ds)

The `cutter_pair_index` dimension is averaged out, which results in the plot below. This shows which fraction of the cutter pairs is gapped for each point in $(B, V)$ space.

In [ ]:
tgp.plot.one.plot_2w_th_avg(ds, vmax=None)

To convert this array of fraction to a boolean array, were `True` is gapped and `False` is gapless, we threshold the array again.
We require that the system is gapped for at least 50\% of the cutter value pairs.

In [ ]:
tgp.one.set_gapped(ds, th_2w_p=0.5)
tgp.plot.one.plot_gapped(ds)

## Extract the ZBPs from $3\omega$ signal

We extract whether ZBPs occur based on the $3\omega$ component.
This works because the 3rd derivative of the conductance informs us about the curvature of the conductance with respect to bias voltage.
We first plot the data.

In [ ]:
lim_3w = 1e9

tgp.one.print_lim(ds.L_3w, lim_1w)
tgp.one.print_lim(ds.R_3w, lim_1w)

tgp.plot.one.plot_3w(ds, vmin=-lim_3w, vmax=lim_3w)

In [ ]:
tgp.plot.one.plot_3w_at(ds, cutter_pair_index=0, lim=lim_3w)  # change the cutter_pair_index to see different plots

To go to a boolean array of ZPBs, we threshold the $3\omega$ data below a negative number, this corresponds with the blue areas in the plots above.
The result of the thresholding operation, and its resulting ZBPs, is plotted below.

In [ ]:
tgp.one.set_3w_th(ds, th_3w=1e7)
tgp.plot.one.plot_3w_th(ds)

The `cutter_pair_index` dimension is averaged out, which results in the plot below. This shows which fraction of the cutter pairs for each point in $(B, V)$ space we get ZBPs.

In [ ]:
tgp.plot.one.plot_zbp(ds)

We require that at least 50\% of the cutter pair values show ZBPs.

In [ ]:
tgp.one.set_3w_tat(ds, th_3w_tat=0.5)  # try, 0.3, 0.5, 0.7
tgp.plot.one.plot_3w_tat(ds)

We cluster the ZPB regions and plot its result.

In [ ]:
tgp.one.set_clusters(ds)
tgp.plot.one.plot_clusters(ds)

## Combining gap and ZBP

Finally, we combine the gap and ZBP arrays to get the following.

In [ ]:
# This variable indicates we include `clusters_n` of the largest clusters in the plots
clusters_n = 11
tgp.plot.one.plot_stage_1(ds, clusters_n)
tgp.plot.one.plot_stage_1_gapped_clusters(ds, clusters_n)

We can plot the interesting regions from most promising to least promising (based on its size.)

In [ ]:
zoomin_ranges = tgp.one.get_zoomin_ranges(ds, clusters_n, zoomin_V_height=0.01)
tgp.plot.one.plot_zoomed_clusters(ds, zoomin_ranges, clusters_n)

Alternatively, we can directly see the numerical values of the regions.

In [ ]:
# Get bounding boxes and area of the clusters
tgp.common.cluster_infos(
    ds["clusters"],
    pct_box=5,  # Margin around box of at least 5%
    min_margin_box=(0.1, 0.002), # and at least 0.1 T and 0.002 V
)